In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error
from sklearn.model_selection import train_test_split, cross_validate
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
PROJECT_PATH = Path(os.path.dirname(os.path.dirname(os.path.abspath('movie-dataset (Gráficos)'))))
DADOS_PATH = PROJECT_PATH / 'dados' / 'saida' / 'BTC-USD.csv'

In [3]:
df = pd.read_csv(DADOS_PATH, usecols=range(1, 7), engine='python')
df

,Open,High,Low,Close,Adj Close,Volume
0,314.846008,315.149994,281.082001,281.082001,281.082001,33054400
1,281.145996,287.230011,257.612000,264.195007,264.195007,55629100
2,265.084015,278.341003,265.084015,274.473999,274.473999,43962800
3,274.610992,287.553009,272.696014,286.188995,286.188995,23245700
4,286.076996,298.753998,283.079010,294.337006,294.337006,24866800
...,...,...,...,...,...,...
3099,30086.187500,30796.250000,30057.203125,30445.351563,30445.351563,13180860821
3100,30441.353516,31256.863281,29600.275391,30477.251953,30477.251953,26387306197
3101,30471.847656,30641.289063,30328.865234,30590.078125,30590.078125,9086606733
3102,30587.269531,30766.140625,30264.019531,30620.769531,30620.769531,10533418042


In [4]:
y = df['Low']
x = df.drop(columns=['Low'], axis=1)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.6)

In [6]:
model = lgb.LGBMRegressor()
model.fit(x_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000088 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 1862, number of used features: 5
[LightGBM] [Info] Start training from score 13994.630754


LGBMRegressor()

In [7]:
y_expected = y_test
y_pred = model.predict(x_test)

In [8]:
print(r2_score(y_expected, y_pred))

0.9991770951318674


In [9]:
print(mean_squared_log_error(y_expected, y_pred))

0.0008140694103667436


In [10]:
cv = cross_validate(model, x, y, cv=5)
print(cv)
print('Test Score: ', cv['test_score'])
print('Test Score (mean): ', cv['test_score'].mean())

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000120 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2483, number of used features: 5
[LightGBM] [Info] Start training from score 16991.400900
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000102 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2483, number of used features: 5
[LightGBM] [Info] Start training from score 15937.905955
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the o

In [11]:
train_rmse = np.sqrt(mean_squared_error(y_train, model.predict(x_train)))
val_rmse = np.sqrt(mean_squared_error(y_test, model.predict(x_test)))

print(f'Treinamento RMSE: {train_rmse}')
print(f'Validação RMSE: {val_rmse}')

# Plotar as curvas de aprendizado
plt.figure(figsize=(10, 6))
plt.plot(model.steps['model_lgbm'].evals_result_, label='Treinamento')
plt.plot(model.evals_result_['valid_1']['rmse'], label='Validação')
plt.xlabel('Número de iterações')
plt.ylabel('RMSE')
plt.title('Curvas de Aprendizado')
plt.legend()
plt.show()

Treinamento RMSE: 288.5381389184461
Validação RMSE: 436.2187458064987


AttributeError: 'LGBMRegressor' object has no attribute 'steps'

<Figure size 1000x600 with 0 Axes>

In [ ]:
model.steps['model_lgbm'].evals_result_

Faça com que